<a href="https://colab.research.google.com/github/zakihir0/jigsaw---Agile-Community-Rules-Classification/blob/main/Jigsaw_%7C_Llama3_1_8B_Instruct_Training_one_epoch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

jigsaw_agile_community_rules_path = kagglehub.competition_download('jigsaw-agile-community-rules')

print('Data source import complete.')


100%|██████████| 695k/695k [00:00<00:00, 38.5MB/s]

Extracting files...
Data source import complete.


In [ ]:
!pip install -U unsloth
!pip install -U torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 100.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch, os, warnings
from datasets import Dataset
from transformers import TrainingArguments, DataCollatorForSeq2Seq, TrainerCallback
from trl import SFTTrainer
import pandas as pd
import numpy as np

warnings.filterwarnings("ignore")

# =======================
# Config
# =======================
max_seq_length = 2048
dtype = None
load_in_4bit = True
SEED = 3407
BATCH_EVAL = 16       # 評価（ロジット計算）バッチサイズ
EVAL_STEPS = 50       # 学習中に test でバリデーションする間隔（steps）

# =======================
# Model & Tokenizer
# =======================
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = SEED,
    use_rslora = False,
    loftq_config = None,
)

device = "cuda" if torch.cuda.is_available() else "cpu"

# =======================
# Data loading
# =======================
system_prompt = (
    "You are given a comment on reddit and a rule. "
    "Your task is to classify whether the comment violates the rule. Only respond Yes/No."
)

# 例: すばやい動作確認用に train は 100 行だけ使用
train_df = pd.read_csv(os.path.join(jigsaw_agile_community_rules_path, "train.csv"))
test_df  = pd.read_csv(os.path.join(jigsaw_agile_community_rules_path, "test.csv"))

user_prompt = (
    "Subreddit: r/{subreddit}\n"
    "Rule: {rule}\n"
    "Examples:\n"
    "1) {positive_example_1}\nViolation: Yes\n\n"
    "2) {negative_example_1}\nViolation: No\n\n"
    "3) {negative_example_2}\nViolation: No\n\n"
    "4) {positive_example_2}\nViolation: Yes\n"
    "Comment:\n{body}\n"
    "Violation: "
)

LABEL_COL = "rule_violation"
CLASSES = ["No", "Yes"]

# =======================
# 共通ユーティリティ（train/test 両方まとめる）
# =======================
def build_conversations(df, system_prompt, user_prompt,
                        label_col=LABEL_COL,
                        include_assistant_if_labeled=True):
    """train/test 共通。ラベルがあれば assistant を付与（損失計算用）。"""
    has_label = (label_col in df.columns)
    convs = []
    for r in df.itertuples(index=False):
        msg_user = user_prompt.format(
            subreddit=getattr(r, "subreddit", ""),
            rule=getattr(r, "rule", ""),
            positive_example_1=getattr(r, "positive_example_1", ""),
            positive_example_2=getattr(r, "positive_example_2", ""),
            negative_example_1=getattr(r, "negative_example_1", ""),
            negative_example_2=getattr(r, "negative_example_2", ""),
            body=getattr(r, "body", ""),
        )
        convo = [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": msg_user},
        ]
        if include_assistant_if_labeled and has_label:
            y = int(getattr(r, label_col))
            convo.append({"role": "assistant", "content": CLASSES[y]})
        convs.append(convo)
    return convs, has_label

def to_text_dataset_from_convs(convs, tokenizer, add_generation_prompt=False):
    texts = [
        tokenizer.apply_chat_template(
            c, tokenize=False, add_generation_prompt=add_generation_prompt
        )
        for c in convs
    ]
    return Dataset.from_dict({"text": texts})

def make_sft_and_eval_sets(train_df, test_df, tokenizer,
                           system_prompt, user_prompt,
                           label_col=LABEL_COL,
                           eval_from_test=True,
                           eval_dummy_n=8):
    """学習(train)・評価(eval=test)・生成評価用(test)を一括準備。"""
    # train（常に assistant 付き）
    train_convs, _ = build_conversations(
        train_df, system_prompt, user_prompt,
        label_col=label_col, include_assistant_if_labeled=True
    )
    train_ds = to_text_dataset_from_convs(train_convs, tokenizer, add_generation_prompt=False)

    # eval は test を使う（ラベル無ければダミー）
    if eval_from_test:
        test_convs_loss, test_has_label = build_conversations(
            test_df, system_prompt, user_prompt,
            label_col=label_col, include_assistant_if_labeled=True
        )
        if test_has_label:
            eval_ds = to_text_dataset_from_convs(test_convs_loss, tokenizer, add_generation_prompt=False)
        else:
            eval_ds = train_ds.select(range(min(eval_dummy_n, len(train_ds))))
    else:
        eval_ds = None

    # 生成/Yes確率評価用（assistant 無し）
    test_convs_gen, _ = build_conversations(
        test_df, system_prompt, user_prompt,
        label_col=label_col, include_assistant_if_labeled=False
    )
    test_texts_for_gen = to_text_dataset_from_convs(
        test_convs_gen, tokenizer, add_generation_prompt=True
    )["text"]

    # test ラベル（あれば後のスコアリングに使用）
    y_test = test_df[label_col].astype(int).values if (label_col in test_df.columns) else None

    return train_ds, eval_ds, test_texts_for_gen, y_test

# ---- データセット作成（ここで train/test を一括処理）----
train_ds, eval_ds, test_texts, test_labels = make_sft_and_eval_sets(
    train_df, test_df, tokenizer, system_prompt, user_prompt,
    label_col=LABEL_COL, eval_from_test=True, eval_dummy_n=8
)

# =======================
# Yes/No verbalizer & スコアリング
# =======================
def build_verbalizer_ids(tokenizer):
    cands_yes = ["Yes", " yes", "Yes.", " yes.", "YES", " YES"]
    cands_no  = ["No",  " no",  "No.",  " no.",  "NO",  " NO"]
    def first_token_ids(cands):
        ids = set()
        for w in cands:
            t = tokenizer(w, add_special_tokens=False)["input_ids"]
            if len(t) > 0: ids.add(t[0])
        return sorted(list(ids))
    yes_ids, no_ids = first_token_ids(cands_yes), first_token_ids(cands_no)
    if len(yes_ids)==0: yes_ids=[tokenizer(" Yes", add_special_tokens=False)["input_ids"][0]]
    if len(no_ids)==0:  no_ids =[tokenizer(" No",  add_special_tokens=False)["input_ids"][0]]
    return yes_ids, no_ids

YES_IDS, NO_IDS = build_verbalizer_ids(tokenizer)

@torch.no_grad()
def yes_probability(texts, batch_size=BATCH_EVAL):
    model.eval()
    probs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        enc = tokenizer(
            batch, return_tensors="pt",
            padding=True, truncation=True, max_length=max_seq_length
        ).to(device)
        out = model(**enc)
        last_logits = out.logits[:, -1, :]                           # [B, V]
        soft = torch.nn.functional.softmax(last_logits, dim=-1)      # [B, V]
        p_yes = soft[:, YES_IDS].sum(dim=-1)
        p_no  = soft[:, NO_IDS].sum(dim=-1)
        p = (p_yes / (p_yes + p_no + 1e-12)).clamp(0,1).detach().cpu().numpy()
        probs.append(p)
        del enc, out
        if device == "cuda":
            torch.cuda.synchronize()
    return np.concatenate(probs, axis=0)

from sklearn.metrics import (
    roc_auc_score, log_loss, accuracy_score,
    precision_recall_fscore_support, confusion_matrix
)
def score_binary(name, y_true, y_prob, print_cm=True):
    auc = roc_auc_score(y_true, y_prob)
    best_thr, best_f1 = 0.5, -1.0
    thrs = np.linspace(0.05, 0.95, 19)
    for t in thrs:
        y_pred = (y_prob >= t).astype(int)
        p, r, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)
        if f1 > best_f1: best_f1, best_thr, best_p, best_r = f1, t, p, r
    y_pred = (y_prob >= best_thr).astype(int)
    acc = accuracy_score(y_true, y_pred)
    ll  = log_loss(y_true, np.clip(y_prob, 1e-6, 1-1e-6))
    print(f"[{name}] AUC={auc:.5f}  LogLoss={ll:.5f}  Acc={acc:.5f}  F1={best_f1:.5f}  P={best_p:.5f}  R={best_r:.5f}  Thr*={best_thr:.2f}")
    if print_cm:
        cm = confusion_matrix(y_true, y_pred, labels=[0,1])
        print(f"[{name}] Confusion Matrix (rows=true, cols=pred):\n{cm}")
    return {"auc":auc, "logloss":ll, "acc":acc, "f1":best_f1, "thr":best_thr}

# =======================
# Train (SFT) + Test ベース評価
# =======================
class TestEvalCallback(TrainerCallback):
    """eval_steps ごとに test を使って先述スコアで評価し、ログ出力"""
    def on_evaluate(self, args, state, control, **kwargs):
        probs = yes_probability(test_texts, batch_size=BATCH_EVAL)
        if test_labels is not None:
            metrics = score_binary(f"LLM (TEST@step {int(state.global_step)})",
                                   test_labels, probs, print_cm=False)
            # Trainer のログにも載せておく（TensorBoard等で可視化）
            kwargs["model"].trainer.log({f"test_{k}": v for k,v in metrics.items()})
        else:
            print(f"[LLM (TEST@step {int(state.global_step)})] sample probs:", probs[:5])

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,          # ← 共通化した train セット
    eval_dataset = eval_ds,            # ← test 由来（ラベル無ならダミー）
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc = 4,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = EVAL_STEPS,
        eval_strategy = "steps",    # ★ eval は常に test ベース
        eval_steps = EVAL_STEPS,
        save_strategy = "no",
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = SEED,
        output_dir = "outputs",
        report_to = "none",
    ),
    callbacks=[TestEvalCallback()],
)

from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>",
    response_part    = "<|start_header_id|>assistant<|end_header_id|>",
)

trainer_stats = trainer.train()

# === 最終 epoch 後：メトリクスを表示（probsは表示しない） ===
final_probs = yes_probability(test_texts, batch_size=BATCH_EVAL)

if test_labels is not None:
    # test にラベルがある → TEST 指標を表示
    _ = score_binary("LLM (TEST final)", test_labels, final_probs, print_cm=True)
else:
    # test にラベルが無い → train から hold-out を作って指標を表示
    from sklearn.model_selection import train_test_split

    if LABEL_COL in train_df.columns and len(train_df) > 10:
        tr_df, va_df = train_test_split(
            train_df, test_size=0.2, random_state=SEED,
            stratify=train_df[LABEL_COL] if train_df[LABEL_COL].nunique() == 2 else None
        )
        # assistant なし（生成用）の会話テキストを作成
        va_convs_gen, _ = build_conversations(
            va_df, system_prompt, user_prompt,
            label_col=LABEL_COL, include_assistant_if_labeled=False
        )
        va_texts = to_text_dataset_from_convs(
            va_convs_gen, tokenizer, add_generation_prompt=True
        )["text"]

        va_probs = yes_probability(va_texts, batch_size=BATCH_EVAL)
        _ = score_binary("LLM (VALID final via hold-out)", va_df[LABEL_COL].astype(int).values, va_probs, print_cm=True)
    else:
        print("[INFO] testにラベルが無く、trainからの評価用分割も作れないため、最終メトリクスは算出できません。")

# =======================
# Save merged adapter
# =======================
# model.save_pretrained_merged("llama-8b-instruct-jigsaw", tokenizer)
# print("Saved to ./llama-8b-instruct-jigsaw")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.3: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files:  25%|██▌       | 1/4 [06:24<19:13, 384.38s/it]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files:  50%|█████     | 2/4 [11:48<11:38, 349.14s/it]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
from google.colab import runtime
runtime.unassign()